In [1]:
# import sys
# sys.path.append("/storage/vbutoi/projects/")
# sys.path.append("/storage/vbutoi/libraries/")
# sys.path.append("/storage/vbutoi/projects/UniverSeg/")
# sys.path.append("/storage/vbutoi/projects/DiSeg/")
# sys.path.append("/storage/vbutoi/projects/SlurmLite/")

# Random Imports
import os

# Change as appropriate to point to the path(s) containing v3.1-midslice-v2
os.environ['DATAPATH'] = ':'.join((
       '/storage',
       '/storage/megamedical'
))
log_root_dir = '/storage/vbutoi/scratch'

%load_ext autoreload
%autoreload 2
%load_ext yamlmagic

In [2]:
from IonPy.util.config import check_missing

def validate_cfg(cfg):
    # It's usually a good idea to do a sanity check of
    # inter-related settings or force them manually
    check_missing(cfg)        
    return cfg

In [3]:
%%yaml default_cfg 

experiment:
  seed: 42
    
dataloader:
  batch_size: 1
  num_workers: 0
  pin_memory: False 

optim: 
  _class: torch.optim.Adam
  lr: 1.0e-5
  
train:
  epochs: 10000 # With heavy aug, you need a lot of epochs, without, only 1000
  eval_freq: 100
  support_size: 8
  accumulate_steps: 1 # Accumulate gradients over multiple steps
  num_samples: 1 # Define how many times to sample the supports.
  l1_lambda: 0.0 # L1 regularization on the logits
  
log:
  checkpoint_freq: 100 
  root: '?'
  metrics:
    dice_score:
      _fn: IonPy.metrics.dice_score
      from_logits: True
      batch_reduction: 'mean' 

loss_func: 
  _class: diseg.experiment.loss.SoftDiceLoss
  from_logits: True
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg  

model:
  _class: diseg.experiment.models.nonparametric.UniverSegScorer
  freeze_backbone: False
  use_label: True
  post_feats: 1
  load_dir: /storage/jjgo/results/omni/2023-03-01_Universeg-SupportSet-Ablation_pt2/20230304_125802-NH10-26180efa296261c9502bf25a551378e5 

sampler:
  _class: diseg.experiment.models.sampler.GumbelSampler
  train_ssize: [1, 32] # Size of the support set for the IC-network.
  val_ssize: 8
  batch_size: 1
  gumbel_hard: False
  temperature: 0.2

incontext_network:
  _class: universeg.experiment.experiment.UniversegExperiment
  load_epoch: max-val_od-dice_score
  load_dir: /storage/jjgo/results/omni/2023-03-01_Universeg-SupportSet-Ablation_pt2/20230304_125802-NH10-26180efa296261c9502bf25a551378e5 

<IPython.core.display.Javascript object>

In [5]:
%%yaml dataset_cfg 

data:
  _class: diseg.experiment.datasets.Segment2D 
  resolution: 128
  label: 0
  task: "WBC/CV/EM/0"
  num_source_examples: [1, 60] # How many examples you can choose from.

<IPython.core.display.Javascript object>

In [6]:
# %%yaml megamedical_cfg
# data:
#   _class: diseg.experiment.datasets.MultiBinarySegment2D
#   preload: False
#   version: v4.2
#   slicing: ['midslice', 'maxslice']
#   min_label_density: 3e-3

In [7]:
# %%yaml datasets_split
# data:
#     train_datasets:
#         - AMOS
#         - BBBC
#         - BrainDevelopment
#         - BrainMetShare
#         - BRATS
#         - BUS
#         - cDEMRIS
#         - CAMUS
#         - CHAOS
#         - CoNSeP
#         - CheXplanation
#         - DRIVE
#         - EOphtha
#         - FeTA
#         - Feto_Plac
#         - FLARE
#         - HMC_QU
#         - I2CVB
#         - IDRID
#         - ISBI
#         - ISLES
#         - KiTS
#         - LGGFlair
#         - LITS
#         - LUNA
#         - MCIC
#         - MSD_BrainTumour
#         - MSD_Colon
#         - MSD_Heart
#         - MSD_HepaticVessel
#         - MSD_Hippocampus
#         - MSD_Liver
#         - MSD_Lung
#         - MSD_Pancreas
#         - MSD_Prostate
#         - MSD_Spleen
#         - OASIS
#         - OCTA500
#         - PROMISE12
#         - PPMI
#         - ROSE
#         - SegTHOR
#         - Synapse_Abdomen
#         - Synthetic1000
#         - VerSe
#         - WMH
#         - WORD
#     val_datasets:
#         - ACDC
#         - PanDental
#         - SCD
#         - SpineWeb
#         - STARE 
#         - TUCC
#         - WBC

In [8]:
%%yaml lite_aug_cfg

- RandomAffine:
    p: 0.5
    degrees: [0, 360]
    translate: [0, 0.2]
    scale: [0.8, 1.1]
- RandomVariableElasticTransform:
    p: 0.75
    alpha: [1, 2] 
    sigma: [6, 8]
- RandomHorizontalFlip:
    p: 0.5
- RandomVerticalFlip:
    p: 0.5

<IPython.core.display.Javascript object>

In [9]:
%%yaml heavy_aug_cfg

- RandomBrightnessContrast:
    p: 0.5
    brightness: [-0.1, 0.1]
    contrast: [0.8, 1.2]
- RandomVariableGaussianBlur:
    p: 0.5
    kernel_size: 5
    sigma: [0.1, 1.1]
- RandomVariableGaussianNoise:
    p: 0.5
    mean: [0, 0.05]
    std: [0, 0.05]
- RandomSharpness:
    sharpness: 5
    p: 0.5
- RandomAffine:
    p: 0.5
    degrees: [0, 360]
    translate: [0, 0.2]
    scale: [0.8, 1.1]
- RandomVariableElasticTransform:
    p: 0.75
    alpha: [1, 2] 
    sigma: [6, 8]
- RandomHorizontalFlip:
    p: 0.5
- RandomVerticalFlip:
    p: 0.5

<IPython.core.display.Javascript object>

In [10]:
%%yaml callbacks_cfg

callbacks:
  epoch:
    - IonPy.callbacks.ETA
    - IonPy.callbacks.JobProgress
    - IonPy.callbacks.TerminateOnNaN
    - IonPy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

<IPython.core.display.Javascript object>

In [11]:
# Need to define the experiment name
exp_name = 'debugging'
#exp_name = '05-30-23_WeDontNeedNoEducation'

# Create the ablation options
options = {
    'log.root': [f'{log_root_dir}/{exp_name}'],
    'model.use_label': [True, False]
}

In [12]:
from IonPy.util import dict_product, Config
import copy

# Setup augmentation lists
light_augmentations = sum([copy.deepcopy(lite_aug_cfg)], start=[])

# Assemble base config
base_cfg = Config(default_cfg).update(model_cfg).update(dataset_cfg).update(callbacks_cfg)

cfgs = []
for cfg_update in dict_product(options):
    cfg = base_cfg.update(cfg_update)
    cfg = cfg.set('augmentations', light_augmentations)
    cfg = validate_cfg(cfg)
    cfgs.append(cfg)

In [13]:
len(cfgs)

2

## Run the jobs

In [14]:
# Submit cell
from diseg.experiment.experiment import DisegExperiment
from slite import SliteRunner

# List the available gpus for a machine
available_gpus = ['0', '1'] 

# Configure Slite Object
srunner = SliteRunner(task_type=DisegExperiment, 
                      exp_name=exp_name, 
                      available_gpus=available_gpus)

## Debug Station

In [15]:
srunner.run_exp(cfgs[0])

Running DisegExperiment("/storage/vbutoi/scratch/debugging/20230531_145310-9Q6A-637f9ed8b373707577767d5208504941")
---
augmentations:
- RandomAffine:
    degrees:
    - 0
    - 360
    p: 0.5
    scale:
    - 0.8
    - 1.1
    translate:
    - 0
    - 0.2
- RandomVariableElasticTransform:
    alpha:
    - 1
    - 2
    p: 0.75
    sigma:
    - 6
    - 8
- RandomHorizontalFlip:
    p: 0.5
- RandomVerticalFlip:
    p: 0.5
callbacks:
  epoch:
  - IonPy.callbacks.ETA
  - IonPy.callbacks.JobProgress
  - IonPy.callbacks.TerminateOnNaN
  - IonPy.callbacks.ModelCheckpoint:
      monitor: dice_score
      phase: val
data:
  _class: diseg.experiment.datasets.Segment2D
  label: 0
  num_source_examples:
  - 1
  - 60
  resolution: 128
  task: WBC/CV/EM/0
dataloader:
  batch_size: 1
  num_workers: 0
  pin_memory: false
experiment:
  seed: 42
incontext_network:
  _class: universeg.experiment.experiment.UniversegExperiment
  load_dir: /storage/jjgo/results/omni/2023-03-01_Universeg-SupportSet-Ablation

OutOfMemoryError: CUDA out of memory. Tried to allocate 240.00 MiB (GPU 0; 11.90 GiB total capacity; 533.54 MiB already allocated; 47.62 MiB free; 626.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Submit Config to Long Term Jobs

In [ ]:
#srunner.submit_exps(cfgs)

In [ ]:
# print(srunner.jobs[0].stderr())